<h1>Section 2.1.4: A Python implementation of the simplex algorithm</h1>

In this worksheet we develop a step by step implementation of the simplex algorithm, following section 2.1 in the text.  We use <code>numpy</code> for matrix operations and <code>pandas</code> to display tableaux with labels.

Since numpy does not have built-in row reduction procedures, we define them in the first code block in the worksheet.

In [0]:
import numpy as np
import pandas as pd

In [0]:
def SwapRow(arrayA,rowi,rowj): #interchange rows i and j
    arrayB=arrayA.copy()
    arrayB[rowi]=arrayA[rowj]
    arrayB[rowj]=arrayA[rowi]
    return arrayB
def AddRow(arrayA,rowi,rowj,s): #add s * row j to row i
    arrayB=arrayA.copy()
    arrayB=arrayB.astype(float) #to avoid division errors with integer arrays
    arrayB[rowi]=arrayA[rowi]+s*arrayA[rowj]
    return arrayB
def MultiplyRow(arrayA,rowi,s): #multiply row i by s
    arrayB=arrayA.copy()
    arrayB=arrayB.astype(float) #to avoid division errors with integer arrays
    arrayB[rowi]=s*arrayA[rowi]
    return arrayB
def Pivot(arrayA,rowi,colj): #pivot matrix at row i and column j
    arrayB=arrayA.copy()
    arrayB=MultiplyRow(arrayA,rowi,1/arrayA[rowi,colj])
    for x in range(0,rowi):
        arrayB=AddRow(arrayB,x,rowi,-arrayB[x,colj])
    for x in range(rowi+1,arrayB.shape[0]):
        arrayB=AddRow(arrayB,x,rowi,-arrayB[x,colj])
    return arrayB

Next we set up the tableaux for an LP of the following form.

<center>
maximize $z={\bf c}\cdot {\bf x}$ <br>
subject to <br>
$A{\bf x}\leq {\bf b}$ <br>
${\bf x} \geq {\bf 0}$
</center>

To use this worksheet for another problem, the data in the next code block should be changed by hand.  The following code blocks should be run to define all the necessary procedures.

In [0]:
c = np.array([4,3])
#create a one-dimensional array of objective function coefficients
A = np.array([[1,0],
             [2,2],
             [3,2]])
#create a two-dimensional array of constraint coefficients
b = np.array([8,28,32])
#create a one-dimensional array of constraint bounds

In [0]:
n=len(c)
m=len(b)
#find the number of decision variables and constraints, respectively
x=['x_'+str(i+1) for i in range(n)]
s=['s_'+str(i+1) for i in range(m)]
Labels=['z']+x+s+['RHS']
#create labels for decision and slack variables
def Tableau(M):
    return pd.DataFrame(M,columns=Labels)
#procedure to add column labels to an LPMatrix
LPMatrix = np.block([
    [1,-c,np.zeros(m+1)],
    [np.zeros((m,1)),A,np.identity(m),b[:,np.newaxis]]
])
#create array corresponding to the initial tableau
def RowRatios(M,c):
    for i in range(m):
        if (M[i+1,c]>0.001):
            print("Row", i+1, "Ratio =", M[i+1,-1]/M[i+1,c])
        else:
            print("Row", i+1, "Ratio Undefined")
#row ratio test for LPMatrix M and column c
def Iterate(M,r,c):
    M2=MultiplyRow(M,r,1/M[r,c])
    M2=Pivot(M2,r,c)
    return M2

Now apply the simplex method step by step.  This notebook differs slightly from the text, since this version of <code>Iterate</code> does not automatically overwrite the LPMatrix.  Instead we save each tableau with a new name, although it is possible to overwrite the same name.

In [0]:
M1=LPMatrix
Tableau(M1)

When computing row ratios, we input the tableau and the column we are using.  The $z$ variable corresponds to column 0 (which we never use for these ratios.)  In this example, the decision variable x1 is column 1, x2 is column 2, s1 is column 3, and so on.

In [0]:
RowRatios(M1,1)

When we call Iterate, we input the tableau, then the row, then the column.  The rows are labeled so that the objective function corresponds to row 0.

In [0]:
M2=Iterate(M1,1,1)
Tableau(M2)

In [0]:
RowRatios(M2,2)

In [0]:
M3=Iterate(M2,3,2)
Tableau(M3)

In [0]:
RowRatios(M3,3)

In [0]:
M4=Iterate(M3,2,3)
Tableau(M4)